In [ ]:
# default_exp knowledge_graph
# hide
_FNAME='knowledge_graph'

import unittest
from unittest import mock
from nbdev.export import notebook2script
import os
TESTCASE = unittest.TestCase()
_nbpath = os.path.join(_dh[0], _FNAME+'.ipynb')

In [ ]:
#export
import os
import logging
import atexit
import json
from pathlib import Path

from retrying import retry
logger = logging.getLogger()

from pydantic import BaseModel, Extra
from typing import List, Optional
import neo4j
import neo4j.exceptions as neo4jexceptions

import scylla

neo_uri = os.environ.get('NEO4J_URI', 'localhost:7687')
neo_auth = os.environ.get('NEO4J_AUTH', 'neo4j/staysovryn')
uri = 'bolt://{0}'.format(neo_uri)
logger.info('Connecting to {0}'.format(uri))
NEO4J_VERSION = "3.5.3"

DEFAULT_DRIVER = None

class BoltConnectionUnsuccessful(Exception):
    pass

@retry(wait_fixed=2000, stop_max_attempt_number=10)
def make_driver(uri=uri, auth=neo_auth, container=None):
    '''
    Create a driver for neo4j database.
    uri: str
        The uri to the bolt port of the database
    auth: str
        auth string in the form of user/password
    container: docker container object
        If provider, the uri will be figured out automatically by examining the container.
        uri will be ignored if container is provided
    '''
    if container:
        uri="bolt://0.0.0.0:{}".format(container.port.bolt)
    try:
        return neo4j.GraphDatabase.driver(uri, auth=tuple(neo_auth.split('/')))
    except OSError:
        raise BoltConnectionUnsuccessful('Could not connect to the neo4j bolt server at {}'.format(uri))
    
def default_driver(force_remake=False, **kwargs):
    global DEFAULT_DRIVER
    if force_remake:
        DEFAULT_DRIVER = None
    if DEFAULT_DRIVER is None:
        driver = make_driver(**kwargs)
        atexit.register(driver.close)
        DEFAULT_DRIVER = driver
    else:
        driver = DEFAULT_DRIVER
    return driver

def tearDown():
    global DEFAULT_DRIVER
    if DEFAULT_DRIVER is not None:
        DEFAULT_DRIVER.close()
    DEFAULT_DRIVER = None

def run_neo4j(version=NEO4J_VERSION, run_on_set_ports=False, **kwargs):
    """Spin up a new neo4j in a docker instance if one is not already running.  
    Set the ichordb default driver to point to this test database.
        """
    image = "neo4j:{}".format(version)
    if run_on_set_ports:
        ports = {7687:17687, 7474:17474, 7473:17473}
    else:
        ports = [7687, 7474, 7473]
    
    additional_kwargs = {'environment': {'NEO4J_AUTH': neo_auth}}
    additional_kwargs.update(kwargs)
    neo4j = scylla.run_container(image, ports=ports, 
                          block_until_port_available=7474,
                          **additional_kwargs)
    neo4j = scylla.set_port_name(neo4j, 7687, 'bolt')
    neo4j = scylla.set_port_name(neo4j, 7474, 'browser')
    neo4j = scylla.set_port_name(neo4j, 7473, 'secure_browser')    
    return neo4j

In [ ]:
#export
from sovrynkg.contracts import whatis, Address
from sovrynkg.abi import decode_log


In [ ]:
name = 'sovrynkg'
container = run_neo4j(container_name=name, run_on_set_ports=True)
driver = default_driver(force_remake=True, container=container)
driver

In [ ]:
#export
class Query(object):
    def __init__(self, driver=None):
        self.driver = default_driver() if driver is None else driver
        self.lines = []

    def add(self, line):
        self.lines.append(line)

    def _make_response(self, session, parameters=None):
        return session.run('\n'.join(self.lines), parameters=parameters)
    
    def results(self, parameters=None):
        with self.driver.session() as session:
            resp = self._make_response(session, parameters)
            records = [r for r in resp]
        return records
    
    def data(self, parameters=None):
        """
        Returns all the data
        """
        try:
            with self.driver.session() as session:            
                resp = self._make_response(session, parameters)
                data = resp.data()
                return data
        except TypeError as typeerror:
            raise NoSearchResults('No results found from query {q} with params {p}'.format(
                        q=self.lines, p=parameters)) from typeerror

    def single(self, parameters=None):
        """
        Returns only a single node.  If multiple responses return, you'll get the first one.
        Will not thrown an exception with multiple responses.
        """
        try:
            with self.driver.session() as session:            
                resp = self._make_response(session, parameters)
                data = resp.single()[0]
                return data            
        except TypeError as typeerror:
            raise NoSearchResults('No results found from query {q} with params {p}'.format(
                        q=self.lines, p=parameters)) from typeerror

    def only(self, parameters=None):
        """
        Use if you expect only a single result to come back.  Throws exceptions if there is not exactly one
        resoponse.
        """
        rd = self.data(parameters)
        if len(rd) > 1:
            raise MultipleSearchResults('Multiple entries found for query {q} with params {p}'.format(
                            q=self.lines, p=parameters))
        try:
            return rd[0]
        except IndexError as indexerror:
            raise NoSearchResults('No results found from query {q} with params {p}'.format(
                        q=self.lines, p=parameters)) from indexerror
    def create(self, parameters=None):
        """
        Use to create nodes
        Returns a summary of how many were created
        """
        with self.driver.session() as session:
            resp = self._make_response(session, parameters)
            counters = resp.consume().counters
        return counters


In [ ]:
#export

def add_constraint(nodetype, prop, driver=None):
    #uniqueness constraints automatically create indexes too
    q = Query(driver=driver)
    q.add("CREATE CONSTRAINT ON (node:{nt}) ASSERT node.{prop} IS UNIQUE".format(nt=nodetype, prop=prop))
    #q.add("CREATE CONSTRAINT ON (xact:Transaction) ASSERT xact.tx_hash IS UNIQUE")    
    return q.create()

def add_constraints(driver=None):
    return add_constraint('Block', 'height'), add_constraint('Transaction', 'tx_hash'), add_constraint('Address', 'address')

In [ ]:
#export
def start_fresh(driver=None):
    query = Query(driver)
    query.add("MATCH (n) DETACH DELETE n")
    query.create()
    
    add_constraints(driver=driver)
    return True

In [ ]:
#export
def make_address(address_obj, driver=None):
    q = Query(driver=driver)
    if address_obj.objtype == 'Address':
        q.add("CREATE (addy:Address)")
    else:
        q.add("CREATE (addy:{}:Address)".format(address_obj.objtype))
    asdict = address_obj.dict()
    for attr in ['prefix', 'name', 'address']:
        raw_val = asdict.get(attr)
        if raw_val is not None:
            val = json.dumps(raw_val)
            q.add("SET addy.{attr}={val}".format(attr=attr, val=val))
    return q.create()

def create_address_if_needed(addy:str, driver=None):
    '''
    addy is the address string
    '''
    if addy is None:
        return None
    
    if not addy.startswith('0x'):
        addy = '0x' + addy
    try:
        address_obj = whatis(addy)[0]
    except IndexError:
        address_obj = Address(objtype='Address', prefix='', name='unknown', address=addy)
    try:
        return make_address(address_obj, driver=driver)
    except neo4jexceptions.ConstraintError:
        pass


In [ ]:
#export

#lets assume that blocks must be added in order
def chainheight(driver=None):
    q=Query(driver=driver)
    q.add('match (b:Block) with max(b.height) as maxheight return maxheight')
    return q.only()['maxheight']

class CanOnlyAddBlocksToTheEnd(Exception):
    pass

class Block:
    def __init__(self, height, signed_at, driver=None):
        self.driver=driver
        self.height=height
        self.signed_at = signed_at
        
    def _create(self, prev_block=None):
        q = Query(driver=self.driver)

        if prev_block:
            q.add('MATCH (prevblock:Block) where prevblock.height={}'.format(prev_block))
            q.add('WITH prevblock')
            q.add("MERGE (prevblock)-[:NEXT]->(b:Block)")

        else:
            q.add("CREATE (b:Block)")
        q.add("SET b.height={}".format(int(self.height)))
        q.add('SET b.signed_at="{}"'.format(self.signed_at))
        return q.create()
    
    def create(self):
        prev_block = chainheight(driver=self.driver)
        if (prev_block is not None) and (self.height < prev_block):
            raise CanOnlyAddBlocksToTheEnd("Cannot add block {h} to a chain with height: {pb}".format(pb=prev_block, h=self.height))
        return self._create(prev_block)
    
    @classmethod
    def from_transaction(clz, transaction):
        return clz(height=transaction.block_height, signed_at=transaction.block_signed_at)


In [ ]:
#export 
class Transaction(BaseModel, extra=Extra.forbid):
    block_height:int
    block_signed_at:str
    from_address:str
    from_address_label: Optional[str]
    gas_offered:int
    gas_price:int
    gas_quote:int
    gas_quote_rate:int
    gas_spent:int
    log_events: List
    successful:bool
    to_address:Optional[str]
    to_address_label: Optional[str]
    tx_hash:str
    tx_offset:int
    value:int
    value_quote:int
        
    def _create(self, driver=None):
        q = Query(driver=driver)
        q.add("CREATE (tx:Transaction)")
        asdict = self.dict()
        for attr in ['gas_offered', 'gas_price', 'gas_quote', 'gas_quote_rate', 
                     'gas_spent', 'successful', 
                     'tx_hash', 'tx_offset', 'value', 'value_quote']:
            raw_val = asdict.get(attr)
            if raw_val is not None:
                val = json.dumps(raw_val)
                q.add("SET tx.{attr}={val}".format(attr=attr, val=val))
            
        q.add("WITH tx")
        q = self.link_to_block(query=q)
        q.add("WITH tx")
        q = self.link_to_addresses(query=q)
        
        return q.create()
    
    def link_to_block(self, query=None):
        query = query or Query()
        query.add("MATCH (b:Block) where b.height={}".format(self.block_height))
        query.add("WITH b, tx")
        query.add("MERGE (b)-[:CONTAINS]->(tx)")
        return query
    
    def link_to_addresses(self, query=None):
        query = query or Query()
        query.add('MATCH (from:Address) where from.address="{}"'.format(self.from_address))
        query.add("WITH from, tx")
        query.add("MERGE (from)<-[:FROM]-(tx)")
        if self.to_address:
            query.add("WITH tx")
            query.add('MATCH (toaddy:Address) where toaddy.address="{}"'.format(self.to_address))
            query.add("WITH toaddy, tx")
            query.add("MERGE (tx)-[:TO]->(toaddy)")

        return query        
        
    def create_block_if_needed(self, driver=None):
        block = Block.from_transaction(self)
        try:
            return block.create()
        except neo4jexceptions.ConstraintError:
            pass    
        
    def create(self):
        results = {}
        results['block']=self.create_block_if_needed()
        results['to'] = create_address_if_needed(self.to_address)
        results['from'] = create_address_if_needed(self.from_address)
        results['create'] = self._create()
        events = []
        for event in self.log_events:
            events.append(LogEvent(**event).create())
        results['events'] = events
        return results

In [ ]:
#export
class LogEvent(BaseModel):
    tx_hash: str
    tx_offset: int
    raw_log_data: Optional[str]
    log_offset: int
    raw_log_topics:List[str]
    decoded: Optional[dict]
    sender_address: str
    def _create(self, query):
        q = query
        q.add("CREATE (le:LogEvent)")
        asdict = self.dict()
        
        for attr in ['tx_hash', 'tx_offset', 'raw_log_data', 'log_offset',
                     'raw_log_topics', 'sender_address']:
            raw_val = asdict.get(attr)
            if raw_val is not None:
                val = json.dumps(raw_val)
                q.add("SET le.{attr}={val}".format(attr=attr, val=val))
        return q
    
    def decode(self):
        return decode_log(topics=self.raw_log_topics, data=self.raw_log_data)
    
    def set_decoded(self):
        if not self.decoded:
            led = self.decode()
            if not led:
                self.decoded = {}
                return self.decoded
            
            name = led['abi'].name
            params = [inp.dict() for inp in led['abi'].inputs]
            for index, param in enumerate(params):
                params[index]['value'] = led['decoded'][param['name']]
            signature = led['abi'].signature()['plain']
            decoded = {'name': name, 'signature': signature, 'params':params}
            self.decoded = decoded
        return self.decoded
    
    def get_signature(self):
        #does it already have a decoded block?
        if self.decoded:
            sig = self.decoded['signature']
        else:
            #can I decode it?
            decoded = self.decode()
            if decoded:
                sig = decoded['plain']
            else:
                #just default to the ugly hash string
                sig = self.raw_log_topics[0]
        return sig
    
    def link_to_sender(self, query):
        q = query
        q.add("WITH le")        
        q.add('MATCH (sender:Address) where sender.address="{}"'.format(self.sender_address))
        q.add("WITH sender, le")
        q.add('MERGE (sender)<-[calls:CALLS]-(le)')
        q.add("WITH calls")        
        if self.decoded:
            for param in self.decoded['params']:
                raw_val = param['value']
                attr = param['name']
                if raw_val is not None:
                    val = json.dumps(raw_val)
                    q.add("SET calls.{attr}={val}".format(attr=attr, val=val))
            q.add("SET calls.signature='{}'".format(self.decoded['signature']))
            q.add("SET calls.name='{}'".format(self.decoded['name']))
            
        else:
            q.add("SET calls.signature='{}'".format(self.get_signature()))
        return q
    
    def link_to_transaction(self, query):
        q = query
        q.add("WITH le")
        q.add("MATCH (tx:Transaction) where tx.tx_hash='{}'".format(self.tx_hash))
        q.add("MERGE (tx)-[:HAS_EVENT]->(le)")
        return q
        
    def check_for_duplicate(self, driver):
        q = Query(driver=driver)
        q.add("MATCH (tx:Transaction)-[:HAS_EVENT]-(event:LogEvent)")
        q.add("WHERE tx.tx_hash='{tx}' and event.log_offset={lo}".format(tx=self.tx_hash, lo=self.log_offset))
        q.add("RETURN event")
        return q.data()
        
    def create(self, driver=None, if_not_exists=True):
        self.set_decoded()
        results = []
        if if_not_exists:
            resp = self.check_for_duplicate(driver=driver)
            if resp:
                logger.warning("{txh} already has event {logoff}".format(txh=self.tx_hash, logoff=self.log_offset))
                return None
        
        results.append(create_address_if_needed(self.sender_address))
        
        q = Query(driver=driver)
        q = self._create(query=q)
        q = self.link_to_transaction(query=q)
        q = self.link_to_sender(query=q)        
        #print('\n'.join([l for l in q.lines]))
        results.append(q.create())
        return results

In [ ]:
def test_when_raw_data_is_none():
    smallevent = {'block_signed_at': '2020-09-30T14:43:59Z',
     'block_height': 2742418,
     'tx_hash': '0x4fbd96b8a2094b7233a914bbf3a253d8823c9db8469a5c30daae405535f2e8ec',
     'tx_offset': 0,
     'successful': True,
     'from_address': '0x7be508451cd748ba55dcbe75c8067f9420909b49',
     'from_address_label': None,
     'to_address': None,
     'to_address_label': None,
     'value': '0',
     'value_quote': 0.0,
     'gas_offered': 6800000,
     'gas_spent': 1898951,
     'gas_price': 65000002,
     'gas_quote': 0.04437211534973109,
     'gas_quote_rate': 359.4868469238281,
     'log_events': [{'block_signed_at': '2020-09-30T14:43:59Z',
       'block_height': 2742418,
       'tx_offset': 0,
       'log_offset': 0,
       'tx_hash': '0x4fbd96b8a2094b7233a914bbf3a253d8823c9db8469a5c30daae405535f2e8ec',
       '_raw_log_topics_bytes': None,
       'raw_log_topics': ['0x8be0079c531659141344cd1fd0a4f28419497f9722a3daafe3b4186f6b6457e0',
        '0x0000000000000000000000000000000000000000000000000000000000000000',
        '0x0000000000000000000000007be508451cd748ba55dcbe75c8067f9420909b49'],
       'sender_contract_decimals': None,
       'sender_name': None,
       'sender_contract_ticker_symbol': None,
       'sender_address': '0x5a0d867e0d70fcc6ade25c3f1b89d618b5b4eaa7',
       'sender_address_label': None,
       'sender_logo_url': None,
       'raw_log_data': None,
       'decoded': {'name': 'OwnershipTransferred',
        'signature': 'OwnershipTransferred(indexed address previousOwner, indexed address newOwner)',
        'params': [{'name': 'previousOwner',
          'type': 'address',
          'indexed': True,
          'decoded': True,
          'value': '0x0000000000000000000000000000000000000000'},
         {'name': 'newOwner',
          'type': 'address',
          'indexed': True,
          'decoded': True,
          'value': '0x7be508451cd748ba55dcbe75c8067f9420909b49'}]}}]}
    tx =Transaction(**smallevent)
    le = [LogEvent(**le) for le in tx.log_events]

    TESTCASE.assertDictEqual({'previousOwner': '0x0000000000000000000000000000000000000000',
  'newOwner': '0x7be508451cd748ba55dcbe75c8067f9420909b49'},
                                 le[0].decode()['decoded']
                            )
test_when_raw_data_is_none()

In [ ]:
#export
def create_transactions(transactions:list):
    '''
    transactions: list of Transactions objects
    '''
    results = {}
    for tx in transactions:
        #tx = Transaction(**rawtx)
        result = tx.create()
        results[tx.tx_hash] = result
        print(tx.block_height, end=', ')
    return results

def load_transactions(fname):
    with open(fname) as f:
        results = json.load(f)

    #put the transactions in block number order, starting from the smallest
    #covalent responds with largest first
    transactions = results['data']['items'][::-1]
    txs = [Transaction(**rawtx) for rawtx in transactions]

    txs.sort(key=lambda tx: tx.block_height)
    return txs

def ls(fpath):
    return [p for p in fpath.iterdir()]

def create_all_transactions(fpath, glob):
    fpath = Path(fpath)
    files = list(fpath.glob(glob))
    files.sort()
    for fname in files:
        txs = load_transactions(fname)
        create_transactions(txs)
    return True

#load_all_transactions('protocol_transactions', 'result.*__*')

In [ ]:
#export
from clize import run

def is_main():
    #not running in jupyter notebook and is the main function
    return not globals().get('get_ipython', False) and globals()['__name__'] == '__main__'

def start_log(logger_name=None):
    '''
    Start a log that prints to stdout
    '''
    logger = logging.getLogger(logger_name)
    screen_handler = logging.StreamHandler()
    formatter = logging.Formatter(
            fmt='[%(levelname)s] %(module)s(%(funcName)s)%(asctime)s--> %(lineno)s.\t %(message)s')
    screen_handler.setFormatter(formatter)
    if not logger.handlers:
        logger.addHandler(screen_handler)
    logger.setLevel(logging.DEBUG)
    return logger

def create_knowledge_graph(fpath, *, glob='result.*__*', delete_existing_database=False):
    '''Creates the sovryn knowledge graph
    
    :param fpath: Directory path to the results files
    :param glob: glob pattern for finding results files within the `fpath` directory
    :param delete_existing_database: delete the existing database first?  False by default
    '''
    driver = default_driver(force_remake=True)
    if delete_existing_database:
        start_fresh()
    create_all_transactions(fpath, glob=glob)

if is_main():
    start_log()    
    logger.setLevel(logging.INFO)
    run(create_knowledge_graph)

In [ ]:
notebook2script(_nbpath)

Converted knowledge_graph.ipynb.
